this notebook will be mainly used for the capstone project

In [4]:
import pandas as pd
import numpy as np

In [5]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### part 1

In [6]:
!pip install beautifulsoup4

In [7]:
!pip install lxml

In [8]:
!pip install requests

In [9]:
from bs4 import BeautifulSoup 
import requests

In [10]:
import pandas as pd

In [11]:
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df = pd.DataFrame(df)
df.head()


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [12]:
df.columns = ['Postal Code', 'Borough', 'Neighborhood'] 

df = df[df.Borough != 'Not assigned']


In [13]:
# function to replace '/' with comma
def replace_with_comma(text):
    text = text.replace(' /',',')
    return text

In [14]:
# avoiding SettingWithCopyWarning 
df = df.copy()

# replacing '/' with comma
df['Neighborhood'] = df['Neighborhood'].apply(replace_with_comma)

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
df.shape

(103, 3)

### Part 2

In [16]:
url2="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(url2)
geo_data.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
print(list(df))
print(list(geo_data))

full_table = df.set_index('Postal Code').join(geo_data.set_index('Postal Code'))
full_table = full_table.sample(frac=1).reset_index(drop=True)
full_table.head(10)

['Postal Code', 'Borough', 'Neighborhood']
['Postal Code', 'Latitude', 'Longitude']


,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Christie,43.669542,-79.422564
1,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
2,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
3,North York,Parkwoods,43.753259,-79.329656
4,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
5,North York,"Willowdale, Newtonbrook",43.789053,-79.408493
6,York,Weston,43.706876,-79.518188
7,North York,Downsview,43.728496,-79.495697
8,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304
9,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577


### Part 3

In [18]:
toronto_data=full_table[full_table['Borough'].str.contains("Toronto")]
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Christie,43.669542,-79.422564
2,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
16,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
19,Central Toronto,Roselawn,43.711695,-79.416936
20,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [19]:
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20180604'

SyntaxError: invalid syntax (<ipython-input-19-372fcb1abfa0>, line 1)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [ ]:
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
import os
from sklearn.cluster import KMeans
!pip install folium
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

In [ ]:
neighborhoods_venues_sorted.head()

In [ ]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Capstone Project - The Battle of Neighborhoods

#####  Analyzing Venues and Real Estate Prices of Delhi
###### 1. Introduction  
###### 1.1. Background
The City of New York, is the most populous city in the United States. It is diverse and is the financial capital of USA. It provides lot of business oppourtunities and business friendly environment. It has attracted many different players into the market. A global hub of business and commerce. The city is a major center for banking and finance, retailing, world trade, transportation, tourism, real estate, new media, traditional media, advertising, legal services, accountancy, insurance, theater, fashion, and the arts in the United States.

This also means that the market is highly competitive. As it is highly developed city so cost of doing business is also one of the highest. Thus, any new business venture or expansion needs to be analysed carefully. The insights derived from this analysis will guide investors a good understanding of the business environment which help in strategically targeting the market. This will help in reduction of risk. And the Return on Investment will be reasonable.

###### 1.2. Problem Description
The City of New York is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigration history. Although, it is difficult to survive in such competitive market, it is very important to startegically plan the business. Various factors need to be studied in opening a restaurant business in NY city but the most critical one would be Location.

###### 1.3 Target Audience
Anyone who is interested to start a new restaurant business in NY city.

###### 1.4 Data Collection
The following data is required to answer hypothesis of the problem:

List of Boroughs and neighborhoods of NY with their geodata (latitud and longitud)
List of Subway metro stations in NY with their address location
List of restaurants available in NY area with their addresses and cuisines 
Venues for each NY neighborhood ( than can be clustered)
Venues for subway metro stations, as needed

The data will be used to solve the problem accordingly:

Use Foursquare and geopy data to map top 10 venues for all NY neighborhoods and clustered in groups
Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.

The procesing of these DATA will allow to answer the following key questions that will help make a decision:

what is the cost of rent (per square ft) around a mile radius from each subway metro station?
what is the area of NY with best rental pricing that meets criteria established?
What is the distance from nearst work places ( Park Ave and 53 rd St) and the tentative future home?
What are the venues of the two best places to live? How the prices compare?
How venues distribute among Manhattan neighborhoods and around metro stations?
Are there tradeoffs between size and price and location?
Any other interesting statistical data findings of the real estate and overall data.